In [29]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.stats import multivariate_normal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Загрузим датасет
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Масштабируем данные
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop('target', axis=1))
y = df['target'].values

# Поделим выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=10)

In [42]:
class CustomClassifier:
    
    # Вычислим средние значения (ядра) и матрицы ковариации для каждого класса в данных обучения
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.means = {}
        self.covariances = {}
        
        for cls in self.classes:
            X_cls = X[y == cls]
            self.means[cls] = np.mean(X_cls, axis=0)
            self.covariances[cls] = np.cov(X_cls, rowvar=False)
    
    # Вычислим расстояния от каждого объекта до среднего значения каждого класса и выберем класс с минимальным расстоянием
    def predict(self, X):
        predictions = []
        
        for x in X:
            distances = []
            for cls in self.classes:
                mean = self.means[cls]
                covariance = self.covariances[cls]
                
                euclid_dist = euclidean(x, mean)
                
                stat_dist = multivariate_normal(mean=mean, cov=covariance).pdf(x)
                
                distances.append((cls, euclid_dist, stat_dist))
            
            chosen_class = min(distances, key=lambda t: t[1])[0]
            predictions.append(chosen_class)
        
        return np.array(predictions)

# Обучим модель на обучающих данных и сделаем предсказания на тестовых данных
classifier = CustomClassifier()
classifier.fit(X_train, y_train)
y_pred_custom = classifier.predict(X_test)

In [48]:
# Обучим knn классификатор
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Сделаем предсказания с помощью knn классификатора
y_pred_knn = knn.predict(X_test)

# Вычислим точность (precision), полноту (recall), процент положительных меток (accuracy) для собственно написанного классификатора
precision_custom = precision_score(y_test, y_pred_custom, average='macro')
recall_custom = recall_score(y_test, y_pred_custom, average='macro')
accuracy_custom = accuracy_score(y_test, y_pred_custom)

# Вычислим точность (precision), полноту (recall), процент положительных меток (accuracy) для классификатора k ближайших соседей
precision_knn = precision_score(y_test, y_pred_knn, average='macro')
recall_knn = recall_score(y_test, y_pred_knn, average='macro')
accuracy_knn = accuracy_score(y_test, y_pred_knn)
                          
print(f"Собственно написанный классификатор: Precision - {precision_custom}, Recall - {recall_custom}, Accuracy - {accuracy_custom}")
print(f"k ближайщих соседей: Precision - {precision_knn}, Recall - {recall_knn}, Accuracy - {accuracy_knn}")

Собственно написанный классификатор: Precision - 0.9539473684210527, Recall - 0.9482051282051283, Accuracy - 0.956140350877193
k ближайщих соседей: Precision - 0.987012987012987, Recall - 0.9743589743589743, Accuracy - 0.9824561403508771


In [45]:
from sklearn.metrics import confusion_matrix

# Вычислим матрицу ошибок (confusion matrix) для собственно написанного классификатора
cm_custom = confusion_matrix(y_test, y_pred_custom)

# Вычислим матрицу ошибок (confusion matrix) для классификатора k ближайших соседей
cm_knn = confusion_matrix(y_test, y_pred_knn)

print(f"Confusion matrix для собственно написанного классификатора:\n{cm_custom}")
print(f"Confusion matrix для k ближайших соседей:\n{cm_knn}")

Confusion matrix для собственно написанного классификатора:
[[36  3]
 [ 2 73]]
Confusion matrix для k ближайших соседей:
[[37  2]
 [ 0 75]]
